In [91]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import umap
import scipy.sparse as sparse
from scipy.io import mmwrite
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [109]:
def pickle_reader(path):
    emb_input = pd.read_pickle(path)
    return emb_input

def reformat_pickle(emb_input):
    """Change every dash (-) by underscore to allow datasets merging"""
    reformatted_emb = {}
    for k, v in emb_input.items():
        new_k = k.replace('-', '_')
        v = v
        reformatted_emb[new_k] = v
    return reformatted_emb

def reformat_df(ORF2_counts):
    """Change every dash (-) and dot (.) by underscore (_) to allow datasets merging"""
    reformatted_counts_dict = {}
    counts = ORF2_counts['Non_redundant']
    names = ORF2_counts['ID']
    for i in range(len(ORF2_counts)):
        number = counts[i]
        ID = names[i]
        not_dash = ID.replace('-', '_')
        clean = not_dash.replace('.', '_')
        #print(clean)
        reformatted_counts_dict[clean] = number
        #reformatted_counts_dict
    
    reformatted = pd.DataFrame.from_dict(reformatted_counts_dict, orient='index')
    reformatted_counts = reformatted.reset_index()
    reformatted_counts.columns = ['ID', 'Non_redundant']
    return reformatted_counts
    
def pickle_to_matrix(reformatted_emb):    # Reading pickle file containing embeddings
    matrix_inverted = pd.DataFrame(reformatted_emb)  # Create df from emb dict
    matrix = matrix_inverted.transpose()  # Invert rows and columns to make it fit for Anndata object
    #matrix.index = ["dimension" + str(i) for i in range(1, matrix.shape[0]+1)]  # dimension as rows, each entry as a column
    return matrix 

def match_ID(matrix, reformatted_counts):
    """Matching matrix (embs) and IDs from counts file to use embs observations with counts associated (all)"""
   #matched_IDs = matrix.loc[:, matrix.index.isin(reformatted_counts["ID"].values)]
    matched_emb = matrix.loc[matrix.index.isin(reformatted_counts['ID'].values)]
    matched_counts = reformatted_counts.loc[reformatted_counts['ID'].isin(matrix.index)] 
    return matched_emb, matched_counts


In [110]:
# -- Read pickle file containing embeddings and reformat to remove dash that block merge
path = '/Users/leandro/Desktop/ai_data/data/embeddings_v2.pickle'
emb_input = pickle_reader(path)
reformatted_emb = reformat_pickle(emb_input)
#reformatted_emb

In [111]:
# -- Read ORF2_counts file and reformat to remove dash and dots by underscore
path_counts = '/Users/leandro/Desktop/ai_data/data/ORF2p_counts.csv'
ORF2_counts = pd.read_csv(path_counts)
reformatted_counts = reformat_df(ORF2_counts)
#reformatted_counts

In [112]:
# -- Creating matrix from reformatted_emb dict and filtering counts and emb dataset to same length
matrix = pickle_to_matrix(reformatted_emb)
#matrix
matched_emb = match_ID(matrix, reformatted_counts)[0]
#matches_emb
matched_counts = match_ID(matrix, reformatted_counts)[1]
#matched_counts

In [87]:
matrix_with_counts=pd.merge(matrix_emb,matched_counts)
matrix_with_counts.set_index('ID')
matrix_nonames = matrix_with_counts.drop(columns='ID')
final_array = np.array(matrix_nonames)
type(final_array)

numpy.ndarray

In [90]:
pca = PCA(n_components=2)
components = pca.fit(final_array)

[1.00000000e+00 1.07800232e-11]


In [94]:
features = matrix_with_counts['Non_redundant']
#features

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [108]:
df = matrix_with_counts
pca = PCA(n_components=2)
components = pca.fit_transform(final_array)
features = df['Non_redundant']
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter(components, labels=labels, color = features)
#fig.update_traces(diagonal_visible=False)
fig.show()

In [92]:
print(len(matrix.index), len(matched_emb.index), len(matched_counts.index))  # Test number of matchs between datasets

721 720 720


In [ ]:

# Run PCA and visualize data
pca = PCA(n_components=subset_matrix.shape[1])
adata.obsm["X_pca"] = pca.fit_transform(adata.X)
sc.pp.neighbors(adata, n_neighbors=10, use_rep="X_pca")
sc.tl.umap(adata, n_components=2, metric="cosine")

# Import Seurat library
import scanpy as sc

# Load data
adata = sc.read_h5ad('your_file.h5ad')

# Run PCA
sc.pp.pca(adata, use_highly_variable=True)

# Visualize PCA results
sc.pl.pca(adata, color='ORF_nonredundant_counts')

# Select PCs for UMAP
sc.pl.pca_variance_ratio(adata)
sc.tl.pca(adata, n_comps=21)
sc.pl.pca_variance_ratio(adata, log=True)

# Run UMAP
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=21)
sc.tl.umap(adata)

# Visualize UMAP results
sc.pl.umap(adata, color='ORF_nonredundant_counts')


In [ ]:
emb_matrix.dtype.names

Note that some of the code in the original R script, such as saving plots to a PDF file, will need to be modified or removed for use in Python.

In [ ]:
# Import Seurat library
import scanpy as sc
import matplotlib.pyplot as plt

# Define a function to load data
def load_data(filename):
    return sc.read_h5ad(filename)



# Define a function to visualize PCA results
def plot_pca(data):
    sc.pl.pca(data, color='ORF_nonredundant_counts')

# Define a function to select PCs for UMAP
def select_pcs(data):
    sc.tl.pca(data, n_comps=21)
    sc.pl.pca_variance_ratio(data, log=True)
    return data

# Define a function to run UMAP
def run_umap(data):
    sc.pp.neighbors(data, n_neighbors=30, n_pcs=21)
    sc.tl.umap(data)
    return data

# Define a function to visualize UMAP results
def plot_umap(data):
    sc.pl.umap(data, color='ORF_nonredundant_counts')

# Load data
adata = load_data('your_file.h5ad')

# Run PCA
adata = run_pca(adata)

# Visualize PCA results
plot_pca(adata)

# Select PCs for UMAP
adata = select_pcs(adata)

# Run UMAP
adata = run_umap(adata)

# Visualize UMAP results
plot_umap(adata)

# Save UMAP plot as a PDF file
plt.savefig('UMAP_1_2_plot.pdf', format='pdf', dpi=300)

In [ ]:
This code defines functions for loading data, running PCA, visualizing PCA results, selecting PCs for UMAP, running UMAP, visualizing UMAP results, and saving a UMAP plot as a PDF file.
The main code loads the data, runs PCA, visualizes PCA results, selects PCs for UMAP, runs UMAP, visualizes UMAP results, and saves a U